In [1]:
import numpy as np
import os
import torch
import json
from sklearn.preprocessing import OneHotEncoder
from tqdm.auto import tqdm
import glob

c:\Users\caleb\anaconda3\envs\cuda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
READ_FILE = 'hands_valid.json'
RANKMAP = {'A':1, 'T':10, 'J':11, 'Q':12, 'K':13}
SUITMAP = {'c':1, 'd':2, 'h':3, 's':4}

In [14]:
def encode_hand_numerical(card_list):
    rank_list = [card[0] for card in card_list]
    suit_list = [card[1] for card in card_list]
    def encode_rank_numerical(ranks):
        return np.vectorize(lambda x: x if x not in RANKMAP.keys() else RANKMAP[x])(np.array(ranks)).reshape(-1,1)
    def encode_suit_numerical(suits):
        return np.vectorize(lambda x: SUITMAP[x])(np.array(suits)).reshape(-1,1)
    encoded_ranks = encode_rank_numerical(rank_list).astype(int)
    encoded_suits = encode_suit_numerical(suit_list).astype(int)
    return np.concatenate((encoded_ranks, encoded_suits), axis=1)


In [16]:
def encode_hand_hot(card_list):
    suits = 'cdhs'  # clubs, diamonds, hearts, spades
    ranks = 'A23456789TJQK'
    all_cards = [rank + suit for rank in ranks for suit in suits]
    card_encoder = OneHotEncoder(categories=[all_cards])
    return card_encoder.fit_transform(np.array(card_list).reshape(-1,1)).toarray()

In [26]:
def encode_board(card_list, rounds, full_ohe=True):
    '''
    Returns an encoding of the board that hides cards that the dealer hasn't revealed yet

    Parameters:
    card_list: list of all five cards on the board
    rounds: one-hot encoding of the game's rounds. rows equal to number of rounds, 4 columns (for pre-flop, flop, turn, river)
    one_hot_ranks, one_hot_suits: see encode_hand

    Returns:
    submatrix ready to be added to the overall encoding. Number of rows depends on embedding choice, number of columns
    equal to the number of rounds
    '''
    full_board = encode_hand_numerical(card_list)
    board_list = []
    if(torch.is_tensor(rounds)):
        rounds = rounds.numpy().T
    for round in rounds:
        if(round[0] == 1):
            board_list.append(np.zeros(full_board.shape).flatten().reshape(-1, 1))
        elif(round[1] == 1):
            flop = full_board[:3]
            hidden = np.zeros((2, full_board.shape[1]))
            board_list.append(np.concatenate((flop, hidden)).flatten().reshape(-1,1))
        elif(round[2] == 1):
            flop_turn = full_board[:4]
            hidden = np.zeros((1, full_board.shape[1]))
            board_list.append(np.concatenate((flop_turn, hidden)).flatten().reshape(-1,1))
        else:
            board_list.append(full_board.flatten().reshape(-1,1))
    return np.concatenate(board_list, axis=1)

In [5]:
def encode_bets(bets):
    '''
    One-hot encodes the rounds of the game (pre-flop, flop, turn, and river) as well as the many
    actions a SINGLE player has made. i.e., we need to call this once per player
    '''
    round_encoder = OneHotEncoder(categories=[['p', 'f', 't', 'r']])
    action_encoder = OneHotEncoder(categories=[['-', 'B', 'f', 'k', 'b', 'c', 'r', 'A', 'Q', 'K']])
    round_list, action_list = [], []
    for round in bets:
        for action in round['actions']:
            round_list.append(round['stage'])
            action_list.append(action)
    rounds = round_encoder.fit_transform(np.array(round_list).reshape(-1, 1)).toarray()
    actions = action_encoder.fit_transform(np.array(action_list).reshape(-1,1)).toarray()
    return rounds, actions

In [9]:
MAXIMUM_SIZE = 20

In [27]:
with open('hands_valid.json', 'r') as f_in:
    with open('hands_filtered.json', 'w') as f_out:
        for line in tqdm(f_in):
            data = json.loads(line)
            if(data['num_players'] == 2):
                action_count = 0
                for round in data['players'][0]['bets']:
                    action_count += len(round['actions'])
                for round in data['players'][1]['bets']:
                    action_count += len(round['actions'])
                if(action_count <= MAXIMUM_SIZE):
                    f_out.write(line)

0it [00:00, ?it/s]

402126it [00:07, 53209.24it/s]


In [40]:
round_encoder = OneHotEncoder(categories=[['p', 'f', 't', 'r']])
action_encoder = OneHotEncoder(categories=[['-', 'B', 'f', 'k', 'b', 'c', 'r', 'A', 'Q', 'K']])

full_data_list = []
full_target_list = []

with open('hands_filtered.json', 'r') as file:
    i = 0
    for line in tqdm(file):
        if (i > 0): break
        data = json.loads(line)
        player_1 = data['players'][0]['bets']
        player_2 = data['players'][1]['bets']
        mixed_action_list = []
        player_list = []
        round_list = []
        for p1_round, p2_round in zip(player_1, player_2):
            j = 0
            while(True):
                try:
                    mixed_action_list.append(p1_round['actions'][j])
                    player_list.append(0)
                    round_list.append(p1_round['stage'])
                except:
                    break
                try:
                    mixed_action_list.append(p2_round['actions'][j])
                    player_list.append(1)
                    round_list.append(p2_round['stage'])
                except:
                    break
                j += 1
        mixed_action_tensor = torch.tensor(action_encoder.fit_transform(np.array(mixed_action_list).reshape(-1,1)).toarray().T)
        round_tensor = torch.tensor(round_encoder.fit_transform(np.array(round_list).reshape(-1,1)).toarray().T)
        player_tensor = torch.tensor(player_list).reshape(1, len(player_list))
        
        encoded_p1_pocket = encode_hand_numerical(data['players'][0]['pocket_cards']).flatten().reshape(-1,1)
        encoded_p2_pocket = encode_hand_numerical(data['players'][1]['pocket_cards']).flatten().reshape(-1,1)
        repeated_p1_pocket = torch.tensor(np.repeat(encoded_p1_pocket, round_tensor.shape[1], axis=1))
        repeated_p2_pocket = torch.tensor(np.repeat(encoded_p2_pocket, round_tensor.shape[1], axis=1))

        hot_p1_target = torch.tensor(encode_hand_hot(data['players'][0]['pocket_cards']).flatten())
        hot_p2_target = torch.tensor(encode_hand_hot(data['players'][1]['pocket_cards']).flatten())
    
        encoded_board = torch.tensor(encode_board(data['board'], round_tensor))
        
        money_features = np.array([data['players'][player][feature] for feature in ['bankroll', 'action', 'winnings'] 
                                        for player in [0,1]]).reshape(6, 1)
        money_features = torch.tensor(np.repeat(money_features, round_tensor.shape[1], axis=1))
        
        full_tensor_p1 = torch.concat((encoded_board, repeated_p1_pocket, money_features, round_tensor, player_tensor, mixed_action_tensor))
        padded_tensor_p1 = torch.nn.functional.pad(full_tensor_p1, (0, MAXIMUM_SIZE-full_tensor_p1.shape[1]), mode='constant', value=0)

        full_tensor_p2 = torch.concat((encoded_board, repeated_p2_pocket, money_features, round_tensor, player_tensor, mixed_action_tensor))
        padded_tensor_p2 = torch.nn.functional.pad(full_tensor_p2, (0, MAXIMUM_SIZE-full_tensor_p2.shape[1]), mode='constant', value=0)

        full_data_list.append(padded_tensor_p1)
        full_data_list.append(padded_tensor_p2)

        full_target_list.append(hot_p2_target)
        full_target_list.append(hot_p1_target)
conjoined_list = [(x,y) for x,y in zip(full_data_list, full_target_list)]

for i, mini_list in enumerate(torch.utils.data.random_split(conjoined_list, [0.1 for k in range(10)])):
    subfolder = 'training'
    if(i == 8):
        subfolder = 'validation'
    if(i == 9):
        subfolder = 'testing'
    print(f'Iteration {i+1}, Saving to {subfolder}')
    torch.save(list(mini_list), os.path.join('data', subfolder, f'data_list_{i}.pt'))

367055it [12:17, 497.93it/s]


In [55]:
type(conjoined_list[1][0])

torch.Tensor

0
training
1
training
2
training
3
training
4
training
5
training
6
training
7
training
8
validation
9
testing
